## <center>This Model will Attempt to Predict the Type of Bicycle Used from Trip Data</center>

In [25]:
# Eliminating Warnings
import warnings
warnings.filterwarnings('ignore')

In [26]:
# Importing Libraries
import pandas as pd
import numpy as np
import os
from pathlib import Path
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import SMOTE
from sklearn.metrics import balanced_accuracy_score

In [27]:
# Set up data path
data_path = 'data'

In [28]:
# Load base data to dataframe
all_data_file = 'all_data.csv'
all_df = pd.read_csv(os.path.join(data_path, all_data_file))
all_df

,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type
0,5/5/21,Dn UWS Up ES,21.25,2.0,31.0,2.52,8.44,Specialized
1,3/20/20,From Despana,4.28,NaN,40.0,0.67,6.42,Citibike
2,10/3/16,Citibike to Work,1.31,NaN,10.0,0.17,7.86,Citibike
3,8/9/16,Citibike to Work,1.31,NaN,11.0,0.18,7.15,Citibike
4,9/20/16,Citibike to Work Midday,1.30,NaN,10.0,0.17,7.80,Citibike
...,...,...,...,...,...,...,...,...
354,7/28/19,Roosevelt Island,4.25,NaN,39.0,0.65,6.54,Specialized
355,6/27/16,Citibike from Work,0.93,NaN,7.0,0.12,7.97,Citibike
356,7/17/16,Down ES UP WS,17.67,2.0,5.0,2.08,8.48,Specialized
357,8/12/19,CP BL,6.79,NaN,48.0,0.80,8.49,Specialized


In [29]:
# Change NaN values to 0 - only occurs on hours column so this is accurate
all_df['Hours'] = all_df['Hours'].fillna(0)
all_df

,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type
0,5/5/21,Dn UWS Up ES,21.25,2.0,31.0,2.52,8.44,Specialized
1,3/20/20,From Despana,4.28,0.0,40.0,0.67,6.42,Citibike
2,10/3/16,Citibike to Work,1.31,0.0,10.0,0.17,7.86,Citibike
3,8/9/16,Citibike to Work,1.31,0.0,11.0,0.18,7.15,Citibike
4,9/20/16,Citibike to Work Midday,1.30,0.0,10.0,0.17,7.80,Citibike
...,...,...,...,...,...,...,...,...
354,7/28/19,Roosevelt Island,4.25,0.0,39.0,0.65,6.54,Specialized
355,6/27/16,Citibike from Work,0.93,0.0,7.0,0.12,7.97,Citibike
356,7/17/16,Down ES UP WS,17.67,2.0,5.0,2.08,8.48,Specialized
357,8/12/19,CP BL,6.79,0.0,48.0,0.80,8.49,Specialized


In [30]:
# Turning Index into Orig_Index Column
all_df.reset_index(inplace=True)
all_df = all_df.rename(columns = {'index': 'Orig_Index'})
all_df

,Orig_Index,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type
0,0,5/5/21,Dn UWS Up ES,21.25,2.0,31.0,2.52,8.44,Specialized
1,1,3/20/20,From Despana,4.28,0.0,40.0,0.67,6.42,Citibike
2,2,10/3/16,Citibike to Work,1.31,0.0,10.0,0.17,7.86,Citibike
3,3,8/9/16,Citibike to Work,1.31,0.0,11.0,0.18,7.15,Citibike
4,4,9/20/16,Citibike to Work Midday,1.30,0.0,10.0,0.17,7.80,Citibike
...,...,...,...,...,...,...,...,...,...
354,354,7/28/19,Roosevelt Island,4.25,0.0,39.0,0.65,6.54,Specialized
355,355,6/27/16,Citibike from Work,0.93,0.0,7.0,0.12,7.97,Citibike
356,356,7/17/16,Down ES UP WS,17.67,2.0,5.0,2.08,8.48,Specialized
357,357,8/12/19,CP BL,6.79,0.0,48.0,0.80,8.49,Specialized


In [31]:
# Defining fields to use
columns = ['Miles', 'Duration', 'Speed','Type']
# target = ['Type']

In [32]:
# Create Dataframe with desired columns
bike_df = all_df.loc[:,columns].copy()
bike_df

,Miles,Duration,Speed,Type
0,21.25,2.52,8.44,Specialized
1,4.28,0.67,6.42,Citibike
2,1.31,0.17,7.86,Citibike
3,1.31,0.18,7.15,Citibike
4,1.30,0.17,7.80,Citibike
...,...,...,...,...
354,4.25,0.65,6.54,Specialized
355,0.93,0.12,7.97,Citibike
356,17.67,2.08,8.48,Specialized
357,6.79,0.80,8.49,Specialized


In [33]:
# Create Dataframe with desired columns
# Demonstrating different way to do it
# bike_df = all_df[['Miles', 'Duration', 'Speed','Type']]
# bike_df

In [34]:
# Splitting out Features and Target
y = bike_df["Type"]
X = bike_df.drop(columns="Type")

In [35]:
# Splitting out Training and Testing data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [36]:
# Checking training data shape
X_train.shape

(269, 3)

In [37]:
# Checking testing data shape
X_test.shape

(90, 3)

In [38]:
# Creating the Logistic Regression Model
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs', max_iter=200, random_state=1)

In [39]:
# Training the Model
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=200, random_state=1)

In [40]:
# Create Predictions
y_pred = classifier.predict(X_test)

In [41]:
# Get accuracy score
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.9777777777777777


In [42]:
# Convert Predictions to DataFrame
pred_df = pd.DataFrame(y_pred)
pred_df.reset_index()
pred_df = pred_df.rename(columns = {0:'Prediction'})
pred_df

,Prediction
0,Specialized
1,Citibike
2,Specialized
3,Specialized
4,Specialized
...,...
85,Citibike
86,Citibike
87,Citibike
88,Specialized


In [43]:
# Convert Tests to DataFrame
test_df = pd.DataFrame(y_test)
test_df

,Type
191,Specialized
313,Citibike
265,Specialized
69,Specialized
141,Specialized
...,...
232,Citibike
324,Citibike
171,Citibike
159,Specialized


In [44]:
# Making index into a column
test_df.reset_index(inplace=True)
test_df = test_df.rename(columns = {'index': 'Orig_Index'})
test_df

,Orig_Index,Type
0,191,Specialized
1,313,Citibike
2,265,Specialized
3,69,Specialized
4,141,Specialized
...,...,...
85,232,Citibike
86,324,Citibike
87,171,Citibike
88,159,Specialized


In [45]:
# Combining the new test and predition dataframes horizontally for comparison
test_pred_df = pd.concat([test_df, pred_df], axis = 1)
test_pred_df

,Orig_Index,Type,Prediction
0,191,Specialized,Specialized
1,313,Citibike,Citibike
2,265,Specialized,Specialized
3,69,Specialized,Specialized
4,141,Specialized,Specialized
...,...,...,...
85,232,Citibike,Citibike
86,324,Citibike,Citibike
87,171,Citibike,Citibike
88,159,Specialized,Specialized


In [46]:
# Identify records where prediction is wrong
pred_errs_df = test_pred_df.loc[test_pred_df['Type'] != test_pred_df['Prediction']]
pred_errs_df

,Orig_Index,Type,Prediction
11,157,Citibike,Specialized
30,18,Citibike,Specialized


In [47]:
# Merging pred_errs_df with all_df to see details of errors
err_details_df = pd.merge(pred_errs_df, all_df, on=["Orig_Index", "Orig_Index"])
err_details_df

,Orig_Index,Type_x,Prediction,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type_y
0,157,Citibike,Specialized,9/16/19,From Pier 11,5.91,0.0,49.0,0.82,7.24,Citibike
1,18,Citibike,Specialized,9/22/16,Work to Westside,4.26,0.0,29.0,0.48,8.81,Citibike


In [48]:
# Cleaning up err_details_df to eliminate redundancy and improve readability
err_details_df.drop(columns='Type_y', inplace=True)
err_details_df.rename(columns = {'Type_x' : 'Type'}, inplace=True)
err_details_df

,Orig_Index,Type,Prediction,Date,Destination,Miles,Hours,Minutes,Duration,Speed
0,157,Citibike,Specialized,9/16/19,From Pier 11,5.91,0.0,49.0,0.82,7.24
1,18,Citibike,Specialized,9/22/16,Work to Westside,4.26,0.0,29.0,0.48,8.81
